In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split as tts

In [ ]:
df0  = pd.read_csv('RAT01.csv',header=0,index_col='No.')
df1 = pd.read_csv('RAT02.csv',header=0,index_col='No.')
df2 = pd.read_csv('RAT03.csv',header=0,index_col='No.')
df3 = pd.read_csv('RAT04.csv',header=0,index_col='No.')
df4 = pd.read_csv('RAT05.csv',header=0,index_col='No.')
df5 = pd.read_csv('RAT06.csv',header=0,index_col='No.')
df6 = pd.read_csv('RAT07.csv',header=0,index_col='No.')
df7 = pd.read_csv('RAT08.csv',header=0,index_col='No.')
df8 = pd.read_csv('P3.csv',header= 0 , index_col='No.',encoding='gbk')
df9 = pd.read_csv('P4.csv',header= 0 , index_col='No.',encoding='gbk')
df10 = pd.read_csv('P2.csv',header= 0 , index_col='No.',encoding='gbk')

In [ ]:
df8.head()

In [ ]:
import pandas as pd
dfs = []  # List to store the DataFrames

for i in range(1, 9):
   file_name = f'RAT0{i}.csv'  # Construct the file name
   df = pd.read_csv(file_name)  # Read the CSV file into a DataFrame
   df['Target'] = file_name[:-4]# Add an 'Target' column with values as filename without extection
   df.drop('No.', axis=1, inplace=True)  # Drop the 'No.' column
   dfs.append(df)  # Append the DataFrame to the list
df_merge = pd.concat(dfs)
#make index column as 'No.'
# Print the first few rows of the DataFrame where 'Malicious' is equal to 1
df_merge.index.name = 'No.'
df_merge['Info'].head()

# dfs =[df,df1,df2,df3,df4,df5,df6,df7]
# df_merge = pd.concat(dfs,ignore_index=True)

In [ ]:
df_merge.shape

In [ ]:
df_merge['Malicious'] = 1
df8['Target'] = 'Benign'
df9['Target'] = 'Benign'
df10['Target'] = 'Benign'
df8['Malicious'] = 0
df9['Malicious'] = 0
df10['Malicious'] = 0
df_merge.head()


In [ ]:
#Now cut down the size of df_merge to 413081 rows
df_merge = df_merge.sample(n=413081, random_state=1)    


In [ ]:
df_merge.shape

In [ ]:
df_merge = pd.concat([df_merge,df8,df9,df10],ignore_index=True)
df_merge.index.name = 'No.'
df_merge.dropna()

In [ ]:
src = []
dst = []
for i in range(len(df_merge)):
    if df_merge['Malicious'][i] == 1:
        src.append(df_merge['Source'][i])
        dst.append(df_merge['Destination'][i])


In [ ]:
from collections import Counter

counter = Counter(src)
counter.most_common()
element_to_remove = '10.8.0.61'
counter.subtract({element_to_remove: counter[element_to_remove]}) # removing your own ip address as source
counter1 = Counter(dst)
counter1.most_common()
counter1.subtract({element_to_remove: counter[element_to_remove]}) # removing your own ip address as destination


In [ ]:
counter1

In [ ]:
import re
Source_ports = []
Destination_ports= []
for i in range(len(df_merge['Info'])):
    text = str(df_merge['Info'][i])
    # Use a regular expression to extract the desired part
    match = re.search(r'(\d+\s{2}\>\s{2}\d+)', text)

    if match:
        result = match.group(1)
        result=result.split('  ')
        if '>' in result:
            result.remove('>')
        t = int(result[0])
        Source_ports.append(t)
        t1 = int(result[1])
        Destination_ports.append(t1)
        #This is a Source ip and Source port number tuple
    else:
        Source_ports.append(0)
        Destination_ports.append(0)

In [ ]:
df_merge['Source_port'] = Source_ports
df_merge['Destination_port'] = Destination_ports
df_merge.head()

In [ ]:
Y = df_merge['Malicious']
X = df_merge.drop('Malicious',axis=1)


In [ ]:
X.Source.value_counts().sort_values(ascending=False).head(50)

In [ ]:
top_10 = [x for x in X.Source.value_counts().sort_values(ascending=False).head(50).index]
top_10 

In [ ]:
for label in top_10:
    X[label]=np.where(X['Source']==label,1,0)
X[['Source']+top_10].head(5)

In [ ]:
top_10 = [x for x in X.Destination.value_counts().sort_values(ascending=False).head(50).index]
top_10                                                                                  

In [ ]:
for label in top_10:
    X[label]=np.where(X['Destination']==label,1,0)
X[['Destination']+top_10].head(5)

In [ ]:
top_10 = [x for x in X.Protocol.value_counts().sort_values(ascending=False).head(50).index]
top_10

In [ ]:
for label in top_10:
    X[label]=np.where(X['Protocol']==label,1,0)
X[['Protocol']+top_10].head(5)

In [ ]:
X.drop(['Source','Destination','Protocol','Info','Target'],axis=1,inplace=True)
X.shape

In [ ]:
X

In [ ]:
df_merge.head()

In [ ]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(sampling_strategy='auto',random_state=42)
# x_res,y_res = sm.fit_resample(X,Y)

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test = tts(X.values,Y.values,test_size=.3,random_state=128)

In [ ]:
# Assuming your DataFrame is named df_merge
df_merge.reset_index(drop=True, inplace=True)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# AdaBoostClassifier

In [ ]:
# Applying AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=3)
ab.fit(x_train, y_train)
ab.score(x_test, y_test)


In [ ]:
#ploting confusion matrix for AdaBoost
import seaborn as sns
sns.heatmap(cmat, annot=True, fmt='d', cmap='Blues', xticklabels=clf.classes_, yticklabels=clf.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix AdaBoost')
plt.show()


In [ ]:
#classification_report for AdaBoost
from sklearn.metrics import classification_report


y_pred_dt = ab.predict(x_test)
print("classification_report of AdaBoost\n")
print(classification_report(y_test,y_pred_dt))


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score

# Binarize the output for binary classification
y_bin = label_binarize(y_test, classes=[0, 1])

# Compute ROC curve and ROC area for each class
fpr, tpr, _ = roc_curve(y_bin, ab.predict_proba(x_test)[:, 1])  # Assuming binary classification
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
#F1 Score for AdaBoost
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred_dt, average='weighted')
print(f'F1 Score of AdaBoost: {f1}')
